In [157]:
import numpy as np 
import pandas as pd
import re
import tensorflow as tf
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [158]:
data = pd.read_csv('/kaggle/input/2023-american-vehicle-prices/prices_clean.csv')

In [159]:
data

,year,make,model,price
0,2023,Honda,Elevate,14530.0
1,2023,Honda,e,37140.0
2,2023,Honda,e Advance,37140.0
3,2023,Honda,Pilot Elite AWD,52030.0
4,2023,Honda,Pilot EX-L 8-Passenger AWD,42310.0
...,...,...,...,...
1165,2023,Tesla,S Performance,104990.0
1166,2023,Tesla,X Performance,120990.0
1167,2023,Tesla,Model 3 Long Range,51990.0
1168,2023,Tesla,Model X Plaid,119990.0


In [160]:
target = data['price']

In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1170 non-null   int64  
 1   make    1170 non-null   object 
 2   model   1170 non-null   object 
 3   price   1170 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 36.7+ KB


In [162]:
stop_words = stopwords.words('english')
def process_text(text):
    text = re.sub(r'\d+',' ',text)
    text = text.split()
    text = ' '.join([word for word in text if word.lower().strip() not in stop_words])
    return text
text = data['model'].apply(process_text)
data = data.drop('model',axis=1)

In [163]:
tokenizer = Tokenizer(num_words = 597)
tokenizer.fit_on_texts(text)
text = tokenizer.texts_to_sequences(text)
max_length = np.max(list(map(lambda x: len(x),text)))

In [164]:
text = pad_sequences(text,maxlen=max_length,padding='post')

In [165]:
data = data.drop('year',axis=1)

In [166]:
data.shape

(1170, 2)

In [167]:
dummie = pd.get_dummies(data['make']).astype(np.int)
data = pd.concat([data,dummie],axis=1)
data = data.drop('make',axis=1)

/tmp/ipykernel_32/531418158.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data['make']).astype(np.int)


In [168]:
y = data['price']
x = data.drop('price',axis=1)

In [169]:
text_train , text_test , x_train , x_test, y_train , y_test = train_test_split(text,x,y,train_size=0.746,random_state=13)

In [170]:
x.shape

(1170, 40)

In [171]:
input_text = tf.keras.Input(shape=(max_length,))
embedding = tf.keras.layers.Embedding(
    input_dim=597,
    output_dim=16,
    input_length=max_length
)(input_text)

flatten = tf.keras.layers.Flatten()(embedding)

input_other = tf.keras.Input(shape=(x.shape[1],))
x = tf.keras.layers.Dense(32,activation='relu')(input_other)
x_end = tf.keras.layers.Dense(32,activation='relu')(x)

concat = tf.keras.layers.concatenate([flatten,x_end])
output = tf.keras.layers.Dense(1,activation='linear')(concat)

model = tf.keras.Model(inputs=[input_text,input_other],outputs=output)

model.compile(
    optimizer='adam',
    loss='mse'
)

model.fit(
    [text_train,x_train],
    y_train
    ,validation_split=0.2
    ,batch_size=32,
    epochs=10,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True
    )
)

Epoch 1/10
22/22 [==============================] - 1s 9ms/step - loss: 228771921920.0000 - val_loss: 184361484288.0000
Epoch 2/10
22/22 [==============================] - 0s 3ms/step - loss: 228771774464.0000 - val_loss: 184361353216.0000
Epoch 3/10
22/22 [==============================] - 0s 3ms/step - loss: 228771610624.0000 - val_loss: 184361091072.0000
Epoch 4/10
22/22 [==============================] - 0s 4ms/step - loss: 228771250176.0000 - val_loss: 184360681472.0000
Epoch 5/10
22/22 [==============================] - 0s 4ms/step - loss: 228770627584.0000 - val_loss: 184360042496.0000
Epoch 6/10
22/22 [==============================] - 0s 5ms/step - loss: 228769742848.0000 - val_loss: 184359108608.0000
Epoch 7/10
22/22 [==============================] - 0s 4ms/step - loss: 228768464896.0000 - val_loss: 184357666816.0000
Epoch 8/10
22/22 [==============================] - 0s 4ms/step - loss: 228766466048.0000 - val_loss: 184355749888.0000
Epoch 9/10
22/22 [======================

### https://www.kaggle.com/datasets/eringill/2023-american-vehicle-prices